# Intro to Deep Learning
#### Jordan DeKraker & Haider Al-Tahan

This is the practical component of this tutorial will focus on hands-on experience with Tensorflow. Topics will include:

1.   Introduction to Tensorflow 2.0 and Keras. 
2.   Train and evaluate  of dense and convolutional neural networks for real problems.
3.   An open session for exploring and adapting existing NNs or for helping attendees build their own networks.

You can use [Google Colaboratory]('http://colab.research.google.com') notebook file which is a [Jupyter notebook]('https://jupyter.org/') with free limited access to a GPU. Jupyter notebooks allows python programs to run directly in the browser which can be great way to learn and use TensorFlow.

If you are using Google colab notebook and want to utilize the free GPU for training, do the following:

1. At the top-left of the of the menu bar, Select `Runtime` > `Change runtime type`. 
2. Click on `Hardware accelerator` > `GPU`.
3. At the top-right of the menu bar, select `CONNECT` to initialize your python interpreter.


# INSTALLATION
Note: you can also use this notebook locally on your machine. Just make sure that you have TensorFlow 2.0.
When installing TensorFlow on your local machine, you also need to make sure that you have the right CPU or GPU version depending your machine.
* If you have a GPU with CUDA and Cudnn or using google colab installed then you just do `pip install tensorflow-gpu==2.0.0` in your terminal.
* If you don't have CUDA and Cudnn installed but have a GPU then follow [these instructions]('https://www.tensorflow.org/install/gpu').
* Otherwise you can use your CPU and install TensorFlow by running `pip install tensorflow==2.0.0` in your terminal.

In [ ]:
# Without GPU
# !pip install tensorflow==2.0.0
# With GPU 
!pip install tensorflow-gpu==2.0.0



